We'll start by importing all the same libraries as last time.
Feel free to add others!

In [8]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
from collections import Counter

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

Here's an example of loading some of the kuzushiji-49 data set.
It will only work after you've run download_data.py on the command line.
This is from the medium-difficulty Japanese literature data set.
You're welcome to try working with the easier or harder data sets instead.

In [115]:
x_data = np.load("kmnist-train-imgs.npz")['arr_0']
y_data = np.load("kmnist-train-labels.npz")['arr_0']
shuffle_index = np.random.shuffle(np.arange(x_data.shape[0]))
x_data = x_data[shuffle_index:]
y_data = y_data[shuffle_index:]

x_train = x_data[:50000]
y_train = y_data[:50000]
x_validate = x_data[50000:]
y_validate = y_data[50000:]
x_test = np.load("kmnist-test-imgs.npz")['arr_0']
y_test = np.load("kmnist-test-labels.npz")['arr_0']

print(x_test.shape)
print(x_validate.shape)
print(x_train.shape)

from collections import Counter
print(Counter(y_train))
print(Counter(y_validate))
print(Counter(y_test))

(10000, 28, 28)
(10000, 28, 28)
(50000, 28, 28)
Counter({3: 5034, 9: 5032, 8: 5026, 1: 5019, 0: 4991, 5: 4991, 7: 4987, 4: 4982, 2: 4972, 6: 4966})
Counter({6: 1034, 2: 1028, 4: 1018, 7: 1013, 0: 1009, 5: 1009, 1: 981, 8: 974, 9: 968, 3: 966})
Counter({2: 1000, 9: 1000, 3: 1000, 8: 1000, 5: 1000, 6: 1000, 1: 1000, 4: 1000, 7: 1000, 0: 1000})


Now it's up to you to handle the data, design an architecture, choose hyperparameters, and train a neural network to solve this classification task.
This time you are welcome to try convolutional layers or other connectivities.

In [116]:
x_train_norm = x_train / 255
x_validate_norm = x_validate / 255
x_test_norm = x_test / 255

y_train_vec = tf.keras.utils.to_categorical(y_train)
y_validate_vec = tf.keras.utils.to_categorical(y_validate)
y_test_vec = tf.keras.utils.to_categorical(y_test)

In [117]:
x_train_norm = x_train_norm.reshape((x_train_norm.shape[0], x_train_norm.shape[1], x_train_norm.shape[2], 1))
x_validate_norm = x_validate_norm.reshape((x_validate_norm.shape[0], x_validate_norm.shape[1], x_validate_norm.shape[2], 1))
x_test_norm = x_test_norm.reshape((x_test_norm.shape[0], x_test_norm.shape[1], x_test_norm.shape[2], 1))
x_train_norm.shape
x_validate_norm.shape

(10000, 28, 28, 1)

In [118]:
neural_net = tf.keras.Sequential()
neural_net.add(layers.Conv2D(filters = 10, kernel_size = (3,3), strides = (1,1), activation = 'relu', input_shape=(28,28, 1)))
neural_net.add(layers.Conv2D(filters = 10, kernel_size = (3,3), strides = (1,1), activation = 'relu'))
neural_net.add(layers.Flatten())
neural_net.add(layers.Dense(64, activation="relu"))
neural_net.add(layers.Dense(10, activation="linear"))
neural_net.compile(loss=losses.MeanSquaredError(), optimizer=optimizers.Adam(learning_rate=0.001), metrics=[metrics.CategoricalAccuracy()])
neural_net.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 26, 26, 10)        100       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 24, 24, 10)        910       
_________________________________________________________________
flatten_23 (Flatten)         (None, 5760)              0         
_________________________________________________________________
dense_49 (Dense)             (None, 64)                368704    
_________________________________________________________________
dense_50 (Dense)             (None, 10)                650       
Total params: 370,364
Trainable params: 370,364
Non-trainable params: 0
_________________________________________________________________


In [119]:
neural_net.fit(x_train_norm, y_train_vec, batch_size=50, epochs=5, validation_data=(x_validate_norm, y_validate_vec))
validation_set_predictions = neural_net.predict(x_validate_norm)

Epoch 1/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0230 - categorical_accuracy: 0.9085 - val_loss: 0.0132 - val_categorical_accuracy: 0.9554
Epoch 2/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0108 - categorical_accuracy: 0.9673 - val_loss: 0.0103 - val_categorical_accuracy: 0.9646
Epoch 3/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0083 - categorical_accuracy: 0.9779 - val_loss: 0.0091 - val_categorical_accuracy: 0.9689
Epoch 4/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0068 - categorical_accuracy: 0.9840 - val_loss: 0.0086 - val_categorical_accuracy: 0.9729
Epoch 5/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0058 - categorical_accuracy: 0.9877 - val_loss: 0.0085 - val_categorical_accuracy: 0.9740


In [120]:
accuracy = tf.keras.metrics.CategoricalAccuracy()
accuracy.update_state(y_validate_vec, validation_set_predictions)
accuracy.result().numpy()

0.974

In [121]:
y_validate_vec
print(validation_set_predictions.shape)
np.argmax(validation_set_predictions, axis = 1)

(10000, 10)


array([0, 8, 5, ..., 0, 4, 9])

In [122]:
test_set_predictions = neural_net.predict(x_test_norm)
accuracy.update_state(y_test_vec, test_set_predictions)
accuracy.result().numpy()

0.9515

Because both the validation accuracy and the test accuracy are similar, we say the model generalizes well to new data and does not overfit to the data. 